# Opening Up America Again

On Thursday, April 16th, the federal government released guidelines for reopening economies for states (and businesses, and individuals) to follow. 

https://www.whitehouse.gov/openingamerica/

These guidelines suggest reopening in 3 phases, each phase more open than the last. To pass from one phase to another, states and localities must meet gating criteria and preparedness milestones such as having adequate testing and contact tracing in place and having adequate healthcare system capacity.

The gating criteria include having a downward trend in cases (or where testing is increasing, a downward trend in cases per test). This notebook examines what states are meeting this criteria.

In [12]:
import datetime
from pprint import pprint
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
from scipy.special import gamma
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_regression
import time
import timeit

from coronavirus.data import load_covidtracking_state_data, add_derived_values_cols

%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Load Data

In [11]:
states_df = (load_covidtracking_state_data(download=False, cache=True)
             .pipe(add_derived_values_cols))
states_df.groupby('entity').last().head()

,date,cases,deaths,tests,population,cases_per_million,cases_per_day,cases_per_day_7day_avg,cases_per_day_3day_avg,cases_per_million_per_day,...,cases_per_test_per_day,cases_per_test_per_day_3day_avg,cases_per_day_3day_avg_tests_per_day_3day_avg_ratio_14day_ratio,cases_per_test_per_day_3day_avg_14day_ratio,deaths_per_test,deaths_per_day_3day_avg_tests_per_day_3day_avg_ratio,deaths_per_test_per_day,deaths_per_test_per_day_3day_avg,deaths_per_day_3day_avg_tests_per_day_3day_avg_ratio_14day_ratio,deaths_per_test_per_day_3day_avg_14day_ratio
entity,,,,,,,,,,,,,,,,,,,,,
Alabama,2020-04-28,6687.0,242.0,75827.0,4903185,1363.807403,188.0,208.000000,183.333333,38.342424,...,0.102732,0.225483,2.125597,0.636004,0.003191,0.006692,0.010929,8.731431e-03,4.180002,2.736260e+00
Alaska,2020-04-28,351.0,9.0,17089.0,731545,479.806437,6.0,3.142857,4.000000,8.201819,...,0.007203,0.020777,0.194547,1.541866,0.000527,0.000000,0.000000,1.156482e-18,0.000000,1.797174e-15
Arizona,2020-04-28,6948.0,293.0,67438.0,7278717,954.563833,232.0,242.428571,222.666667,31.873749,...,0.259218,0.158578,1.169933,1.310639,0.004345,0.005477,0.020112,8.006559e-03,0.849122,1.086692e+00
Arkansas,2020-04-28,3111.0,52.0,40671.0,3017804,1030.882059,94.0,126.285714,94.000000,31.148478,...,0.439252,0.198424,1.082671,1.560680,0.001279,0.001910,0.009346,3.923457e-03,0.812643,2.181710e+00
California,2020-04-28,45031.0,1809.0,577608.0,39512223,1139.672653,1567.0,1681.428571,1298.000000,39.658614,...,0.064755,0.054518,0.424046,0.134654,0.003132,0.002208,0.002231,2.273710e-03,0.446426,9.273911e-02


## What States Have a 14-Day Downward Trend in Cases (or Cases/Test)?

In [8]:
# cases per day 3day avg. 14 day lag ratio.
# state report:
# - (cases per day 3day avg) 14day ratio
# - (cases per day 3day avg / tests per day 3day avg) 14 day ratio

df = states_df
(df.groupby('entity').last()
#  .sort_values(by='cases_per_day_3day_avg_tests_per_day_3day_avg_ratio_14day_ratio')
#  .sort_values(by='cases_per_day_3day_avg_14day_ratio')
 .sort_values(by='cases_per_day_3day_avg_14day_ratio')
 .loc[:, ['date',
          'cases_per_day_3day_avg_14day_ratio',
          'cases_per_day_3day_avg_tests_per_day_3day_avg_ratio_14day_ratio',
          'deaths_per_test',
          'deaths_per_day_3day_avg_tests_per_day_3day_avg_ratio_14day_ratio',
          'cases_per_day',
          'cases_per_million_per_day',
         ]])


# df.groupby(['entity'])[values_per_million].transform(lambda s: s.rolling(3).mean())
# df.groupby(['entity'])[values_per_million].transform(lambda s: s.rolling(8).apply(lambda w: w.iloc[-1] - w.iloc[0]))

# df.groupby('entity')[]
# df['cases_per_day_3day_avg_tests_per_day']


,date,cases_per_day_3day_avg_14day_ratio,cases_per_day_3day_avg_tests_per_day_3day_avg_ratio_14day_ratio,deaths_per_test,deaths_per_day_3day_avg_tests_per_day_3day_avg_ratio_14day_ratio,cases_per_day,cases_per_million_per_day
entity,,,,,,,
Hawaii,2020-04-28,0.153846,0.254257,0.000548,4.958005,1.0,0.706279
Washington,2020-04-28,0.263158,0.589762,0.004258,4.668947,165.0,21.668065
Montana,2020-04-28,0.272727,0.256615,0.001137,0.940922,2.0,1.871296
Oregon,2020-04-28,0.385496,0.518468,0.001797,0.960668,0.0,0.000000
Maine,2020-04-28,0.423729,47.779661,0.002861,112.760000,17.0,12.646815
Alaska,2020-04-28,0.428571,0.194547,0.000527,0.000000,6.0,8.201819
South Dakota,2020-04-28,0.458564,0.582231,0.000662,inf,68.0,76.865775
Vermont,2020-04-28,0.463415,1.153755,0.003089,0.622420,7.0,11.218146
Louisiana,2020-04-28,0.514628,0.914265,0.011919,1.347437,218.0,46.893883
